# Flowsheet Validator

The overall configuration of a flowsheet can be specified without providing detail within the individual process groups. This is demonstrated in Chapter 4 of the d'Anterroches (2006) thesis where upper case alphabetic characters substitute for process groups. The resulting strings do not form valid SFILES sequences, but are nevertheless useful means of describing flowsheets at a higher level of abstraction.

The following examples are from d'Anterroches (2006):

     AC[<B]DE                                  <== Figure 4.7
     ABC[<J<I<H]D[<L[<K]<M]EFG                 <== Figure 4.8 (w/correction)
     HIJC[<B<A]D[<L[<K]<M]EFG                  <== Figure 4.8 (alternative)
     AB<1CD1E                                  <== Figure 4.9
     OA<1E[P]F[Q]G<2MN[R]J<3K[IH3]L[S]D2CB1    <== Figure 4.10

The following cell demonstrates the parsing and validation of these sequences using the `pyparsing` library.

In [33]:
from pyparsing import Literal, Word, Group
from pyparsing import Optional, OneOrMore, ZeroOrMore
from pyparsing import alphas, nums
from pyparsing import nestedExpr, Forward

prevNode = [None]
prevConn = [None]

class ParseNode():
    def __init__(self, tokens):
        global prevNode
        self.tokens = tokens
        self.assignFields()
    def __str__(self):
        return self.__class__.__name__ + ': ' + str(self.name)
    __repr__ = __str__  
    
class Processgroup(ParseNode):
    def assignFields(self):
        self.name = self.tokens[0]
        self.prevConn = prevConn[-1]
        
class Connector(ParseNode):
    def assignFields(self):
        self.name = str(self.tokens[0])
        prevConn.append(self)  # tentatively add to prev connector list

class Recycle(ParseNode):
    def assignFields(self):
        self.name = self.tokens[0][0]

class Branch(ParseNode):
    def assignFields(self):
        self.name = self.tokens
        
class SFILES(ParseNode):
    def assignFields(self):
        self.name = self.tokens

# parsar grammar
processgroup = Word(alphas.upper(), exact=1)  
processgroup.setParseAction(Processgroup)

# connector
connector = Optional(Literal('<') | Literal('>') , Literal('>'))
connector.setParseAction(Connector)

# recycle
recycle = Group(Word(nums, exact=1))
recycle.setParseAction(Recycle)

# branch
branch = Forward()
branchsequence = connector + processgroup + ZeroOrMore(connector + (recycle | branch | processgroup))
branch = nestedExpr(opener="[", closer="]", content=branchsequence)
branch.setParseAction(Branch)

# sfiles
sfiles = processgroup + ZeroOrMore(connector + (recycle | branch | processgroup))
sfiles.setParseAction(SFILES)


{W:(ABCD...) [{[{"<" | ">"}] {{Group:(W:(0123...)) | nested [] expression} | W:(ABCD...)}}]...}

In [34]:
result = sfiles.parseString('ABCD')

At this stage, `sfiles` is an object from the [`ParserElement`](https://pythonhosted.org/pyparsing/pyparsing.ParserElement-class.html) class from the `pyparsing` library.

In [35]:
tests = """\
    ABC
    A<BC
    A<1BC1D
    AC[<B]DE
    ABC[<J<I<H]D[<L[<K]<M]EFG
    HIJC[<B<A]D[<L[<K]<M]EFG
    AB<1CD1E
    OA<1E[P]F[Q]G<2MN[R]J<3K[IH3]L[S]D2CB1"""

result = sfiles.runTests(tests);


ABC
[SFILES: [Processgroup: A, Connector: ">", Processgroup: B, Connector: ">", Processgroup: C]]


A<BC
[SFILES: [Processgroup: A, Connector: <, Processgroup: B, Connector: ">", Processgroup: C]]


A<1BC1D
[SFILES: [Processgroup: A, Connector: <, Recycle: 1, Connector: ">", Processgroup: B, Connector: ">", Processgroup: C, Connector: ">", Recycle: 1, Connector: ">", Processgroup: D]]


AC[<B]DE
[SFILES: [Processgroup: A, Connector: ">", Processgroup: C, Connector: ">", Branch: [[Connector: <, Processgroup: B]], Connector: ">", Processgroup: D, Connector: ">", Processgroup: E]]


ABC[<J<I<H]D[<L[<K]<M]EFG
[SFILES: [Processgroup: A, Connector: ">", Processgroup: B, Connector: ">", Processgroup: C, Connector: ">", Branch: [[Connector: <, Processgroup: J, Connector: <, Processgroup: I, Connector: <, Processgroup: H]], Connector: ">", Processgroup: D, Connector: ">", Branch: [[Connector: <, Processgroup: L, Branch: [[Connector: <, Processgroup: K]], Connector: <, Processgroup: M]], Connec